# Conjunction AllenNLP BERT

Load the needed packages: 

In [1]:
from allennlp_models.pretrained import load_predictor

/home/elena/anaconda3/lib/python3.9/site-packages/pkg_resources/__init__.py:122: PkgResourcesDeprecationWarning: 4.0.0-unsupported is an invalid version and will not be supported in a future release
  warnings.warn(


In [2]:
import checklist
from checklist.editor import Editor
from checklist.perturb import Perturb
from checklist.test_types import MFT, INV, DIR
from checklist.expect import Expect

In [3]:
from checklist.pred_wrapper import PredictorWrapper

In [4]:
import logging
logging.getLogger('allennlp.common.params').disabled = True 
logging.getLogger('allennlp.nn.initializers').disabled = True 
logging.getLogger('allennlp.modules.token_embedders.embedding').disabled = True 
logging.getLogger('urllib3.connectionpool').disabled = True 
logging.getLogger('allennlp.common.plugins').disabled = True 
logging.getLogger('allennlp.common.model_card').disabled = True 
logging.getLogger('allennlp.models.archival').disabled = True 
logging.getLogger('allennlp.data.vocabulary').disabled = True 
logging.getLogger('cached_path').disabled = True

Creating a prediction by the model

In [5]:
srl_predictor = load_predictor('structured-prediction-srl-bert')
output = srl_predictor.predict("John met Max and smiled.")
output

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


{'verbs': [{'verb': 'met',
   'description': '[ARG0: John] [V: met] [ARG1: Max] and smiled .',
   'tags': ['B-ARG0', 'B-V', 'B-ARG1', 'O', 'O', 'O']},
  {'verb': 'smiled',
   'description': '[ARG0: John] met Max and [V: smiled] .',
   'tags': ['B-ARG0', 'O', 'O', 'O', 'B-V', 'O']}],
 'words': ['John', 'met', 'Max', 'and', 'smiled', '.']}

In [6]:
def predict_srl(data):
    
    pred = []
    for d in data:
        pred.append(srl_predictor.predict(d))
    return pred

predict_and_conf = PredictorWrapper.wrap_predict(predict_srl)

In [7]:
d = ["John met Max and smiled."]
pred = predict_and_conf(d)
pred

([{'verbs': [{'verb': 'met',
     'description': '[ARG0: John] [V: met] [ARG1: Max] and smiled .',
     'tags': ['B-ARG0', 'B-V', 'B-ARG1', 'O', 'O', 'O']},
    {'verb': 'smiled',
     'description': '[ARG0: John] met Max and [V: smiled] .',
     'tags': ['B-ARG0', 'O', 'O', 'O', 'B-V', 'O']}],
   'words': ['John', 'met', 'Max', 'and', 'smiled', '.']}],
 array([1.]))

In [8]:
def format_srl(x, pred, conf, label=None, meta=None):
    
    return pred['verbs'][0]['description']

In [9]:
def get_arg(pred, arg_target='ARG0'):
    # we assume one predicate:
    predicate_arguments = pred['verbs'][0]
    #print(predicate_arguments)
    words = pred['words']
    tags = predicate_arguments['tags']
    
    arg_list = []
    for t, w in zip(tags, words):
        arg = t
        if '-' in t:
            arg = t.split('-')[1]
        if arg == arg_target:
            arg_list.append(w)
    arg_set = set(arg_list)
    return arg_set

In [10]:
def format_srl(x, pred, conf, label=None, meta=None):
    results = []
    predicate_structure = pred['verbs'][0]['description']
        
    return predicate_structure

In [11]:
def found_arg1_people(x, pred, conf, label=None, meta=None):
    
    # people should be recognized as arg1

    people = set([meta['first_name']])
    arg_1 = get_arg(pred, arg_target='ARG1')

    if arg_1 == people:
        pass_ = True
    else:
        pass_ = False
    return pass_


expect_arg1 = Expect.single(found_arg1_people)

In [12]:
def found_arg0_people(x, pred, conf, label=None, meta=None):
    
    # people should be recognized as arg0

    people = set([meta['first_name']])
    arg_0 = get_arg(pred, arg_target='ARG0')

    if arg_0 == people:
        pass_ = True
    else:
        pass_ = False
    return pass_


expect_arg0 = Expect.single(found_arg0_people)

## Test for first predicate

In [13]:
# initialize editor object
editor = Editor()

# create examples
t = editor.template("{first_name} met Max and smiled.", meta=True,nsamples=500, remove_duplicates=True)


In [14]:
with open('/datasets/conjunction_pred1_arg0_bert.txt', 'w') as f:
    print(t.data, file = f)

In [15]:
# initialize a test object
import csv
test = MFT(**t, name = 'detect_arg0_name_default_position', expect=expect_arg0)
output = test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)
i = test.results['preds']
expect_for_i = test.results['expect_results']
with open ('/results/conjunction_pred1_arg0_bert.csv','w') as f:
    writer = csv.writer(f)
    for result, exp in zip(i, expect_for_i):
        
        case = result['words']
        exp = exp
        writer.writerow([case, exp])
        print(case,exp)

Predicting 500 examples
Test cases:      500
Fails (rate):    0 (0.0%)
['Helen', 'met', 'Max', 'and', 'smiled', '.'] [ True]
['Laura', 'met', 'Max', 'and', 'smiled', '.'] [ True]
['Matt', 'met', 'Max', 'and', 'smiled', '.'] [ True]
['Ellen', 'met', 'Max', 'and', 'smiled', '.'] [ True]
['Lisa', 'met', 'Max', 'and', 'smiled', '.'] [ True]
['Johnny', 'met', 'Max', 'and', 'smiled', '.'] [ True]
['Benjamin', 'met', 'Max', 'and', 'smiled', '.'] [ True]
['Albert', 'met', 'Max', 'and', 'smiled', '.'] [ True]
['Janet', 'met', 'Max', 'and', 'smiled', '.'] [ True]
['Harry', 'met', 'Max', 'and', 'smiled', '.'] [ True]
['Alexander', 'met', 'Max', 'and', 'smiled', '.'] [ True]
['Alan', 'met', 'Max', 'and', 'smiled', '.'] [ True]
['Harry', 'met', 'Max', 'and', 'smiled', '.'] [ True]
['Jeff', 'met', 'Max', 'and', 'smiled', '.'] [ True]
['Johnny', 'met', 'Max', 'and', 'smiled', '.'] [ True]
['Jane', 'met', 'Max', 'and', 'smiled', '.'] [ True]
['Elizabeth', 'met', 'Max', 'and', 'smiled', '.'] [ True]
['

In [16]:
# initialize editor object
editor = Editor()

# create examples
t = editor.template("John met {first_name} and smiled.", meta=True,nsamples=500, remove_duplicates=True)
with open('/datasets/conjunction_pred1_arg1_bert.txt', 'w') as f:
    print(t.data, file = f)

# initialize a test object
test = MFT(**t, name = 'detect_arg0_name_default_position', expect=expect_arg1)
output = test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)
i = test.results['preds']
expect_for_i = test.results['expect_results']
with open ('/results/conjunction_pred1_arg1_bert.csv','w') as f:
    writer = csv.writer(f)
    for result, exp in zip(i, expect_for_i):
        
        case = result['words']
        exp = exp
        writer.writerow([case, exp])
        print(case,exp)


Predicting 500 examples
Test cases:      500
Fails (rate):    0 (0.0%)
['John', 'met', 'James', 'and', 'smiled', '.'] [ True]
['John', 'met', 'Thomas', 'and', 'smiled', '.'] [ True]
['John', 'met', 'Patrick', 'and', 'smiled', '.'] [ True]
['John', 'met', 'Charles', 'and', 'smiled', '.'] [ True]
['John', 'met', 'Thomas', 'and', 'smiled', '.'] [ True]
['John', 'met', 'Nicole', 'and', 'smiled', '.'] [ True]
['John', 'met', 'Bob', 'and', 'smiled', '.'] [ True]
['John', 'met', 'Walter', 'and', 'smiled', '.'] [ True]
['John', 'met', 'Steve', 'and', 'smiled', '.'] [ True]
['John', 'met', 'Barbara', 'and', 'smiled', '.'] [ True]
['John', 'met', 'Edith', 'and', 'smiled', '.'] [ True]
['John', 'met', 'Christopher', 'and', 'smiled', '.'] [ True]
['John', 'met', 'Sue', 'and', 'smiled', '.'] [ True]
['John', 'met', 'Sue', 'and', 'smiled', '.'] [ True]
['John', 'met', 'Sam', 'and', 'smiled', '.'] [ True]
['John', 'met', 'Florence', 'and', 'smiled', '.'] [ True]
['John', 'met', 'Alice', 'and', 'smile

In [17]:
# initialize editor object
editor = Editor()

# create examples
t = editor.template("{first_name} met {first_name1} and smiled.", meta=True,nsamples=500, remove_duplicates=True)
with open('/datasets/conjunction_pred1_arg0_2firstname_bert.txt', 'w') as f:
    print(t.data, file = f)

# initialize a test object
test = MFT(**t, name = 'detect_arg0_name_default_position', expect=expect_arg0)
output = test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)
i = test.results['preds']
expect_for_i = test.results['expect_results']
with open ('/results/conjunction_pred1_arg0_2firstname_bert.csv','w') as f:
    writer = csv.writer(f)
    for result, exp in zip(i, expect_for_i):
        
        case = result['words']
        exp = exp
        writer.writerow([case, exp])
        print(case,exp)

Predicting 496 examples
Test cases:      496
Fails (rate):    0 (0.0%)
['Michael', 'met', 'Fred', 'and', 'smiled', '.'] [ True]
['Nancy', 'met', 'Sandra', 'and', 'smiled', '.'] [ True]
['Jonathan', 'met', 'Tom', 'and', 'smiled', '.'] [ True]
['Al', 'met', 'Kevin', 'and', 'smiled', '.'] [ True]
['Sally', 'met', 'Kelly', 'and', 'smiled', '.'] [ True]
['Jessica', 'met', 'Ashley', 'and', 'smiled', '.'] [ True]
['Sandra', 'met', 'Caroline', 'and', 'smiled', '.'] [ True]
['Ellen', 'met', 'Harold', 'and', 'smiled', '.'] [ True]
['Jonathan', 'met', 'Joe', 'and', 'smiled', '.'] [ True]
['Larry', 'met', 'Judy', 'and', 'smiled', '.'] [ True]
['Tim', 'met', 'Roy', 'and', 'smiled', '.'] [ True]
['Robin', 'met', 'Katherine', 'and', 'smiled', '.'] [ True]
['Christine', 'met', 'Barbara', 'and', 'smiled', '.'] [ True]
['Stephen', 'met', 'Carl', 'and', 'smiled', '.'] [ True]
['Caroline', 'met', 'Jerry', 'and', 'smiled', '.'] [ True]
['Tom', 'met', 'Sandra', 'and', 'smiled', '.'] [ True]
['Florence', 'me

In [18]:
def found_arg1_people(x, pred, conf, label=None, meta=None):
    
    # people should be recognized as arg1

    people = set([meta['first_name1']])
    arg_0 = get_arg(pred, arg_target='ARG1')

    if arg_0 == people:
        pass_ = True
    else:
        pass_ = False
    return pass_


expect_arg1 = Expect.single(found_arg1_people)

In [19]:
# initialize editor object
editor = Editor()

# create examples
t = editor.template("{first_name} met {first_name1} and smiled.", meta=True,nsamples=500, remove_duplicates=True)
with open('/datasets/conjunction_pred1_arg1_2firstname2_bert.txt', 'w') as f:
    print(t.data, file = f)

# initialize a test object
test = MFT(**t, name = 'detect_arg0_name_default_position', expect=expect_arg1)
output = test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)
i = test.results['preds']
expect_for_i = test.results['expect_results']
with open ('/results/conjunction_pred1_arg1_2firstname2_bert.csv','w') as f:
    writer = csv.writer(f)
    for result, exp in zip(i, expect_for_i):
        
        case = result['words']
        exp = exp
        writer.writerow([case, exp])
        print(case,exp)

Predicting 498 examples
Test cases:      498
Fails (rate):    0 (0.0%)
['Donald', 'met', 'Thomas', 'and', 'smiled', '.'] [ True]
['Bill', 'met', 'Tony', 'and', 'smiled', '.'] [ True]
['Jerry', 'met', 'Evelyn', 'and', 'smiled', '.'] [ True]
['Joe', 'met', 'Ian', 'and', 'smiled', '.'] [ True]
['Tony', 'met', 'Carl', 'and', 'smiled', '.'] [ True]
['Elaine', 'met', 'Sarah', 'and', 'smiled', '.'] [ True]
['Henry', 'met', 'Annie', 'and', 'smiled', '.'] [ True]
['Emily', 'met', 'Sandra', 'and', 'smiled', '.'] [ True]
['Bill', 'met', 'Sam', 'and', 'smiled', '.'] [ True]
['Wendy', 'met', 'Claire', 'and', 'smiled', '.'] [ True]
['Marie', 'met', 'Alfred', 'and', 'smiled', '.'] [ True]
['Andrew', 'met', 'Amanda', 'and', 'smiled', '.'] [ True]
['Fiona', 'met', 'Rachel', 'and', 'smiled', '.'] [ True]
['Jack', 'met', 'Patricia', 'and', 'smiled', '.'] [ True]
['Mary', 'met', 'Jimmy', 'and', 'smiled', '.'] [ True]
['Kim', 'met', 'Samuel', 'and', 'smiled', '.'] [ True]
['Frances', 'met', 'Brian', 'and',

## Test for second predicate

In [20]:
def format_srl(x, pred, conf, label=None, meta=None):
    
    return pred['verbs'][1]['description']

In [21]:
def get_arg(pred, arg_target='ARG0'):
    # we assume one predicate:
    predicate_arguments = pred['verbs'][1]
    #print(predicate_arguments)
    words = pred['words']
    tags = predicate_arguments['tags']
    
    arg_list = []
    for t, w in zip(tags, words):
        arg = t
        if '-' in t:
            arg = t.split('-')[1]
        if arg == arg_target:
            arg_list.append(w)
    arg_set = set(arg_list)
    return arg_set

In [22]:
def format_srl(x, pred, conf, label=None, meta=None):
    results = []
    predicate_structure = pred['verbs'][1]['description']
        
    return predicate_structure

In [23]:
# initialize editor object
editor = Editor()

# create examples
t = editor.template("{first_name} met Max Mustermann and smiled.", meta=True,nsamples=500, remove_duplicates=True)
# initialize a test object
with open('/datasets/conjunction_pred2_arg0_bert.txt', 'w') as f:
    print(t.data, file = f)

# initialize a test object
test = MFT(**t, name = 'detect_arg0_name_default_position', expect=expect_arg0)
output = test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)
i = test.results['preds']
expect_for_i = test.results['expect_results']
with open ('/results/conjunction_pred2_arg0_bert.csv','w') as f:
    writer = csv.writer(f)
    for result, exp in zip(i, expect_for_i):
        
        case = result['words']
        exp = exp
        writer.writerow([case, exp])
        print(case,exp)

Predicting 500 examples
Test cases:      500
Fails (rate):    0 (0.0%)
['Kim', 'met', 'Max', 'Mustermann', 'and', 'smiled', '.'] [ True]
['Virginia', 'met', 'Max', 'Mustermann', 'and', 'smiled', '.'] [ True]
['Don', 'met', 'Max', 'Mustermann', 'and', 'smiled', '.'] [ True]
['Daniel', 'met', 'Max', 'Mustermann', 'and', 'smiled', '.'] [ True]
['Judith', 'met', 'Max', 'Mustermann', 'and', 'smiled', '.'] [ True]
['Carol', 'met', 'Max', 'Mustermann', 'and', 'smiled', '.'] [ True]
['Alan', 'met', 'Max', 'Mustermann', 'and', 'smiled', '.'] [ True]
['Dave', 'met', 'Max', 'Mustermann', 'and', 'smiled', '.'] [ True]
['Dave', 'met', 'Max', 'Mustermann', 'and', 'smiled', '.'] [ True]
['Rebecca', 'met', 'Max', 'Mustermann', 'and', 'smiled', '.'] [ True]
['Jonathan', 'met', 'Max', 'Mustermann', 'and', 'smiled', '.'] [ True]
['Donald', 'met', 'Max', 'Mustermann', 'and', 'smiled', '.'] [ True]
['Pamela', 'met', 'Max', 'Mustermann', 'and', 'smiled', '.'] [ True]
['Cynthia', 'met', 'Max', 'Mustermann', 

# End of Notebook